In [130]:
%run model_sucrose.ipynb

Knocking out reactions mentioned in the paper by Wu et al (Wu et al., 2019):
- LDH which converts pyruvate to lactate
- PTA which converts Acetyl-CoA to Acetyl phosphate

### Finding knockout targets

#### Finding the LDH reaction:

In [106]:
model.reactions.query("LDH")

[<Reaction BETALDHx at 0x28028be0dc0>,
 <Reaction LDH2 at 0x2802b550190>,
 <Reaction LDH_L at 0x2802b570c70>,
 <Reaction VALDHr at 0x2802b789e50>]

Checking if LDH_L is the one producing lactate

In [107]:
model.reactions.LDH_L.metabolites

{<Metabolite lac__L_c at 0x28028a35280>: -1.0,
 <Metabolite nad_c at 0x28028a2d400>: -1.0,
 <Metabolite h_c at 0x28028a342b0>: 1.0,
 <Metabolite nadh_c at 0x28028a2d6d0>: 1.0,
 <Metabolite pyr_c at 0x2802b33ea90>: 1.0}

In [108]:
model.metabolites.lac__L_c.name

'L-Lactate'

Yes. Seems like LDH_L is the reaction we're looking for.

#### Finding the PTA reaction

In [109]:
model.reactions.query("PTA")

[<Reaction APTA1i at 0x28028b29ca0>,
 <Reaction ASPTA at 0x28028bd4e20>,
 <Reaction GF6PTA at 0x2802b452940>,
 <Reaction PTAr at 0x2802b606fd0>,
 <Reaction PTA2 at 0x2802b606d30>]

Checking if PTAr is the reaction Acetyl phosphate:

In [110]:
model.reactions.PTAr.metabolites

{<Metabolite accoa_c at 0x2802b2d0eb0>: -1.0,
 <Metabolite pi_c at 0x28028a3a760>: -1.0,
 <Metabolite actp_c at 0x280286735e0>: 1.0,
 <Metabolite coa_c at 0x28028673250>: 1.0}

In [111]:
model.metabolites.actp_c.name

'Acetyl phosphate'

Yes. Seems like PTAr is the reaction we're looking for.

### Knocking out reactions

Now we'll try knocking it out each of the reactions and see what happens to the growth and surfactin yield:

Knockout of LDH_L:

In [131]:
with model as model:
    model.reactions.LDH_L.bounds = 0, 0
    print("LDH_L's bounds inside the with statement:")
    print(model.reactions.LDH_L.lower_bound, model.reactions.LDH_L.bounds)
    model.objective=model.reactions.DM_surfactin_c
    surfactin_production = model.optimize().objective_value #flux
    print('Maximum productivity =', surfactin_production, 'mmol/gDW*h')
    maximum_yield = surfactin_production / (-1*(model.reactions.EX_sucr_e.flux)) #yield calculations
    print('Maximum theoretical yield =', maximum_yield, 'mmol-surfactin/mmol-sucrose')

LDH_L's bounds inside the with statement:
0 (0, 0)
Maximum productivity = 0.3436433767512154 mmol/gDW*h
Maximum theoretical yield = 0.12001049195903496 mmol-surfactin/mmol-sucrose


Knockout of PTAr:

In [132]:
with model as model:
    model.reactions.PTAr.bounds = 0, 0
    print("LDH_L's bounds inside the with statement:")
    print(model.reactions.PTAr.lower_bound, model.reactions.PTAr.bounds)
    model.objective=model.reactions.DM_surfactin_c
    surfactin_production = model.optimize().objective_value #flux
    print('Maximum productivity =', surfactin_production, 'mmol/gDW*h')
    maximum_yield = surfactin_production / (-1*(model.reactions.EX_sucr_e.flux)) #yield calculations
    print('Maximum theoretical yield =', maximum_yield, 'mmol-surfactin/mmol-sucrose')

LDH_L's bounds inside the with statement:
0 (0, 0)
Maximum productivity = 0.3436433767512154 mmol/gDW*h
Maximum theoretical yield = 0.11996699542671414 mmol-surfactin/mmol-sucrose


In both cases we see an increase in yield